Here, we will solve problems two ways
1. First using PySpark function 
2. Second using Spark SQL

In [1]:
# First Load all the required library and also Start Spark Session
# Load all the required library
from pyspark.sql import SparkSession

In [2]:
#Start Spark Session
spark = SparkSession.builder.appName("problem7").getOrCreate()
sqlContext = SparkSession(spark)
#Dont Show warning only error
spark.sparkContext.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/02/15 10:13:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
#Load CSV file into DataFrame
transactiondf = spark.read.format("csv").option("header","true").option("inferSchema","true").load("transaction.csv")

In [4]:
#Check Schema of DataFrame
transactiondf.printSchema()

root
 |-- id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- item: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- revenue: integer (nullable = true)



In [10]:
from pyspark.sql.functions import col
transactiondf = transactiondf.withColumn("created_at",col("created_at").cast("date"))

In [12]:
transactiondf.printSchema()

root
 |-- id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- item: string (nullable = true)
 |-- created_at: date (nullable = true)
 |-- revenue: integer (nullable = true)



In [13]:
#Check sample Data 
transactiondf.show()

+---+-------+-------+----------+-------+
| id|user_id|   item|created_at|revenue|
+---+-------+-------+----------+-------+
|  1|    109|   milk|2020-03-03|    123|
|  2|    139|biscuit|2020-03-18|    421|
|  3|    120|   milk|2020-03-18|    176|
|  4|    108| banana|2020-03-18|    862|
|  5|    130|   milk|2020-03-28|    333|
|  6|    103|  bread|2020-03-29|    862|
|  7|    122| banana|2020-03-07|    952|
|  8|    125|  bread|2020-03-13|    317|
|  9|    139|  bread|2020-03-30|    929|
| 10|    141| banana|2020-03-17|    812|
| 11|    116|  bread|2020-03-31|    226|
| 12|    128|  bread|2020-03-04|    112|
| 13|    146|biscuit|2020-03-04|    362|
| 14|    119| banana|2020-03-28|    127|
| 15|    142|  bread|2020-03-09|    503|
| 16|    122|  bread|2020-03-06|    593|
| 17|    128|biscuit|2020-03-24|    160|
| 18|    112| banana|2020-03-24|    262|
| 19|    149| banana|2020-03-29|    382|
| 20|    100| banana|2020-03-18|    599|
+---+-------+-------+----------+-------+
only showing top

In [14]:
# Now we are solving Same problem using Spark SQL 
# Creating Temp Table or HIVE table
transactiondf.createOrReplaceTempView("tmpTransaction")

In [16]:
# Now we have SQL Table and we can write SQL Query on top of that 
# For example by Select on table 
sqlContext.sql("SELECT DISTINCT(a1.user_id) \
                FROM tmpTransaction a1 \
                JOIN tmpTransaction a2 ON a1.user_id=a2.user_id \
                AND a1.id <> a2.id \
                AND DATEDIFF(a2.created_at,a1.created_at) BETWEEN 0 AND 7 \
                ORDER BY a1.user_id;").show()

+-------+
|user_id|
+-------+
|    100|
|    103|
|    105|
|    109|
|    110|
|    111|
|    112|
|    114|
|    117|
|    120|
|    122|
|    128|
|    129|
|    130|
|    131|
|    133|
|    141|
|    143|
|    150|
+-------+

